Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [NHL API](https://gitlab.com/dword4/nhlapi).
<hr>

# NHL Team Timeouts - 2020-21 Season

Nearing the end of the season during playoffs, one of the Tampa Bay's games against Montreal had a pretty early timeout `2021-07-03` the 196th timeout of the season was called by Montreal after falling down by 2 in the 1st period in game 3. The commentators started talking about how while rare, many teams end up losing a game and never even using their timeout at all. 

This stuck with me so I decided to find any potentially interesting fact about timeouts that I could for the 2020-21 season. I chose to stick to just a single season rather then multiple seasons as previously failed coaches challenges lead to a loss of a timeout and didn't want to mix the data.
 
This code below is used to get the `team totals` and the `weekly timeouts`. The weekly timeouts is used to see if the overall percent of timeouts being used increased later into the season (playoffs) and groups games and timeouts by [ISO-8601 Week Number](https://en.wikipedia.org/wiki/ISO_8601). The team totals simply adds up the total games played by each team and how often they called a timeout.

<hr>

In [1]:
import pandas as pd

In [2]:
# load game data and add week number too games
games = pd.read_csv('../data/clean/game_data.csv', parse_dates=['dateTime', 'endDateTime'])
games['week_num'] = games['dateTime'].dt.isocalendar().week

# load timeout data
timeout = pd.read_csv('../data/clean/timeouts_clean.csv')
timeout = timeout[['game_id', 'timeout_team']]
timeout = timeout.rename(columns={"game_id": "pk", "timeout_team": "team"})
timeout['timeout'] = True

In [3]:
id_vars = [x for x in list(games) if not x.endswith('_team')]
df = games.melt(id_vars=id_vars)

df.loc[(df['winner'] == 'home') & (df['variable'] == 'home_team'), 'outcome'] = 'win'
df.loc[(df['winner'] == 'away') & (df['variable'] == 'away_team'), 'outcome'] = 'win'
df['outcome'] = df['outcome'].fillna('lose')

# change value to team
df = df.rename(columns={'value': 'team'})

# merge timeouts with data
df = df.merge(timeout, how='left', on=['pk', 'team'])
df['timeout'] = df['timeout'].fillna(False)

# get team totals
team_totals = (df
    .groupby(['team', 'outcome'], as_index=False)['outcome'].size()
    .pivot_table(index='team', columns='outcome')
    .xs('size', axis=1, drop_level=True)
    .reset_index().rename(columns={'value': 'team'})
    .assign(total=df.groupby(['team']).size().values)
    .assign(timeouts=df.groupby(['team'])['timeout'].sum().values)
)

# get totals by season week num
weekly_timeout = timeout.groupby('pk', as_index=False)['timeout'].sum()
weekly = games.merge(weekly_timeout, how='left', on='pk').fillna(0)
weekly = weekly.groupby('week_num')['timeout'].agg(['size', 'sum']).reset_index()
weekly['size'] = weekly['size'] * 2 # 2 timeouts per game (one each team)
weekly = weekly.rename(columns={'size': 'available_timeouts', 'sum': 'used_timeouts'})

# get percentages for both weekly and teams
weekly['percent_used'] = weekly['used_timeouts'] / weekly['available_timeouts']
team_totals['pct_timeouts'] = team_totals['timeouts'] / team_totals['total']

weekly.to_csv('../data/clean/weekly_timeouts.csv', index=False)
team_totals.to_csv('../data/clean/team_timeouts.csv', index=False)